In [120]:
import torch
from torch import nn 
import nce
import torch.nn.functional as F
import numpy as np

In [130]:
np.__version__

'1.18.5'

In [121]:
import pickle
with open("./SEVIR_IR069_NORMED_CMAPS/gray_CV_data_split_27.pickle", "rb") as pickle_file:
    data_gray = pickle.load(pickle_file)

In [122]:
with open("./SEVIR_IR069_OPT_FLOWS/ir069gray_CV_data_split_27.pickle_flow.pickle", "rb") as pickle_file:
    data_cv = pickle.load(pickle_file)

In [116]:
data_grey[0][1].shape

(49, 192, 192)

In [123]:
data_cv[0][1].shape

(48, 192, 192, 2)

In [ ]:
data_cv[0]

In [113]:
class SevirEmbedderCNN(nn.Module):
    def __init__(self, img_size, embedding_size):
        super(SevirEmbedderCNN, self).__init__()
        self.img_size = img_size
        self.embedding_size = embedding_size
        self.conv1 = nn.Conv2d(2, 2, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(2, 16, 5)
        self.conv3 = nn.Conv2d(16, 16, 5)
        self.conv4 = nn.Conv2d(16, 16, 5)
        self.conv5 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, self.embedding_size)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [132]:
sevirCNN = SevirEmbedderCNN(data[0][1][1].shape, 64)
sevirCNN(torch.from_numpy(np.array(np.moveaxis(data_cv[0][1],-1,1)))).shape

torch.Size([48, 64])

In [133]:
np.array([data[0][1][0:2]]).shape

(1, 2, 192, 192)

In [156]:
class SevirLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SevirLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm_1 = nn.LSTM(self.input_size, self.hidden_size, batch_first=True)
        self.lstm_2 = nn.LSTM(self.hidden_size, self.hidden_size, batch_first=True)
    def forward(self, input):
        return self.lstm_1(input)

In [157]:
sevirLSTM = SevirLSTM(64, 64)

In [158]:
test_embedding = sevirCNN(torch.from_numpy(np.array(np.moveaxis(data_cv[0][1],-1,1))))

In [159]:
test_embedding.shape

torch.Size([48, 64])

In [168]:
sevirLSTM(test_embedding.view(-1,48,64))[0].shape

torch.Size([1, 48, 64])

In [172]:
sevirLSTM(test_embedding.view(-1,48,64))[1][0].shape

torch.Size([1, 1, 64])

In [173]:
sevirLSTM(test_embedding.view(-1,48,64))[1][1].shape

torch.Size([1, 1, 64])

In [ ]:
asd